In [1]:
import pandas as pd
import glob
import hashlib
import sqlite3
import os
from faker import Faker
import random

fake = Faker()

Unire tutti i dati nei CSV files dentro `data` in un singolo Pandas dataframe in memoria.

In [2]:
db_filename = 'snowflake_schema_example.db'
if os.path.exists(db_filename):
    os.remove(db_filename)
cnx = sqlite3.connect(db_filename)
dfs = []
for csv_f in glob.glob("data/*.csv"):
    dfs.append(pd.read_csv(csv_f, index_col=None, header=0))

df = pd.concat(dfs, axis=0, ignore_index=True)
df.dropna(how="any", inplace=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541020e+12,583,Sehr kosmisch,200,1.542240e+12,26.0
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541020e+12,583,The Big Gundown,200,1.542240e+12,26.0
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541020e+12,583,Marry Me,200,1.542240e+12,26.0
5,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540490e+12,597,Blackbird,200,1.542250e+12,61.0
9,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,NextSong,1.540790e+12,602,Best Of Both Worlds (Remastered Album Version),200,1.542260e+12,80.0


Arricchico il dataframe con dati sintetici per avere una demo piu' corposa.

In [3]:
df["song_artist"]=df["song"]+" - "+df["artist"]
df["song_id"] = df["song_artist"].apply(lambda x: hashlib.md5(x.encode('utf-8')).hexdigest())
df.drop(["song_artist"],axis=1,inplace=True)
df["company_name"] = [fake.company() for _ in range(df.shape[0])]
df["company_address"] = [fake.street_address() for _ in range(df.shape[0])]
df["company_suffix"] = [fake.company_suffix() for _ in range(df.shape[0])]
df["companyName_Suffix"] = df["company_name"]+" - "+df["company_suffix"]
df["company_id"] = df["companyName_Suffix"].apply(lambda x: hashlib.md5(x.encode('utf-8')).hexdigest())
df.drop(["companyName_Suffix"],axis=1,inplace=True)
df["length_class"] = ['long' if x >= 232.972605 else 'short' for x in df['length']]
df["file_format"] = [random.choice(['mp3','flac','wav']) for _ in range(df.shape[0])]
df["song_artist_format"]=df["song"]+" - "+df["artist"]+" - "+df["file_format"]
df["meta_id"] = df["song_artist_format"].apply(lambda x: hashlib.md5(x.encode('utf-8')).hexdigest())
df.drop(["song_artist_format"],axis=1,inplace=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,ts,userId,song_id,company_name,company_address,company_suffix,company_id,length_class,file_format,meta_id
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1.542240e+12,26.0,eeac062e56a7e543d7d6e517eab89560,Foster-Vincent,86069 James Way,Group,4df0e20b51a7a66169fdfe001b2a4211,long,mp3,62d86ac0e951e317d57d911567706f71
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1.542240e+12,26.0,525671f443d2425a0583bb03afaa1c28,Russell-Johnson,868 Smith Islands Apt. 994,and Sons,5a5e39680c5254b9dd280d40d2dc13a3,long,wav,c49212847115420b3a280f4f86878386
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,...,1.542240e+12,26.0,8807df8a9a4b422534027b675d40d78a,Watson-Garcia,0359 Edwin Glen Suite 806,Inc,429c8abd3a65ce85dce6fa2349a6a8cf,short,wav,cb600ec94abf04b3d23ae2f46480b528
5,Sony Wonder,Logged In,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",PUT,...,1.542250e+12,61.0,2a4d00ff68a501f27d381152ad471b94,Peterson-Peck,305 John Road Suite 393,Inc,638c294c3ae91456f3837378a3483efe,short,wav,fa0942aea355634f759517d628e101b7
9,Van Halen,Logged In,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",PUT,...,1.542260e+12,80.0,3036905f8ce3396815bf12699d8904a0,"Wilson, Stokes and Davis",98806 Connor Mill,PLC,d01cdc1f3c592143d1d4acf934412374,long,wav,f518a23de6b25079cc6ea8f60cf59ead


Crea un nuovo dataframe per la *fact table*:


In [4]:
fact_df = df[["auth","userId","sessionId","itemInSession","ts","level","song_id"]].drop_duplicates()
fact_df.head()

,auth,userId,sessionId,itemInSession,ts,level,song_id
0,Logged In,26.0,583,0,1.542240e+12,free,eeac062e56a7e543d7d6e517eab89560
1,Logged In,26.0,583,1,1.542240e+12,free,525671f443d2425a0583bb03afaa1c28
2,Logged In,26.0,583,2,1.542240e+12,free,8807df8a9a4b422534027b675d40d78a
5,Logged In,61.0,597,0,1.542250e+12,free,2a4d00ff68a501f27d381152ad471b94
9,Logged In,80.0,602,2,1.542260e+12,paid,3036905f8ce3396815bf12699d8904a0


Crea *dimension tables* base per le rispettive dimensioni:

- song
- user
- session
- time

In [5]:
song_dim = df[["song_id","artist","song","length"]].drop_duplicates()
song_meta_dim = df[["meta_id","song_id","length_class","file_format"]].drop_duplicates()
user_dim = df[["userId","firstName","lastName","gender","location","company_id"]].drop_duplicates()
company_dim = df[["userId","company_id","company_name","company_address","company_suffix"]].drop_duplicates()
session_dim = df[["sessionId","page","itemInSession","method"]].drop_duplicates()
time_dim = df[["ts"]].drop_duplicates()

Arricchisci le *time_ds* con piu' informazioni:

In [6]:
time_dim["timestamp"] = pd.to_datetime(time_dim["ts"],unit="ms")
time_dim["day"]= time_dim["timestamp"].dt.day
time_dim["month"]= time_dim["timestamp"].dt.month
time_dim["year"]= time_dim["timestamp"].dt.year
time_dim["quarter"]= time_dim["timestamp"].dt.quarter
time_dim.head()

,ts,timestamp,day,month,year,quarter
0,1.542240e+12,2018-11-15 00:00:00,15,11,2018,4
5,1.542250e+12,2018-11-15 02:46:40,15,11,2018,4
9,1.542260e+12,2018-11-15 05:33:20,15,11,2018,4
24,1.542270e+12,2018-11-15 08:20:00,15,11,2018,4
39,1.542280e+12,2018-11-15 11:06:40,15,11,2018,4


Salva in un SQLite database.

In [7]:
fact_df.to_sql(name='songplay_fact', con=cnx)
song_dim.to_sql(name='song_dim', con=cnx)
song_meta_dim.to_sql(name='song_meta_dim', con=cnx)
user_dim.to_sql(name='user_dim', con=cnx)
company_dim.to_sql(name='company_dim', con=cnx)
session_dim.to_sql(name='session_dim', con=cnx)
time_dim.to_sql(name='time_dim', con=cnx)

242

Test di lettura:

In [8]:
results = pd.read_sql("""SELECT sp.userId,
                                sp.level,
                                t.timestamp,
                                t.quarter,
                                c.company_name
                        FROM songplay_fact sp INNER JOIN time_dim t ON sp.ts = t.ts
                        INNER JOIN company_dim c ON sp.userId = c.userId""", cnx)
results.head()

,userId,level,timestamp,quarter,company_name
0,26.0,free,1542236400,4,Allen and Sons
1,26.0,free,1542236400,4,Allen-Shelton
2,26.0,free,1542236400,4,"Arellano, Lowe and Burgess"
3,26.0,free,1542236400,4,"Armstrong, Moss and Thomas"
4,26.0,free,1542236400,4,Atkinson Inc
